In [ ]:
%pip install -U ollama openai

# 

# Simple Tool Calling for LLM + GMAIL

In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os
from datetime import datetime

# Email credentials and SMTP settings
smtp_server = "smtp.gmail.com"
port = 587  # For starttls
sender_email = "lucasenkrateia@gmail.com"  # Enter your email
password = os.environ["GMAIL_PWD"]

def send_email(to, subject, body):
    """
    Send an email using the configured sender email and SMTP settings
    
    Args:
        to (str): Email address of the recipient
        subject (str): Subject line of the email
        body (str): Body text of the email
    """
    # Create message
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = to
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))

    # Send email
    try:
        server = smtplib.SMTP(smtp_server, port)
        server.starttls()  # Secure the connection
        server.login(sender_email, password)
        server.sendmail(sender_email, to, message.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        server.quit()

# Example usage
today_date = datetime.now().strftime('%d %B %Y')
email_subject = f"LUCAS IS LEARNING AND TEACHING"
email_body = f"Hi Lucas! You are awesome, don't give up! Today's date is {today_date}."
send_email("lucasbnsoares@hotmail.com", email_subject, email_body)

Email sent successfully!


![](2024-12-02-18-14-58.png)

In [2]:
import ollama


response = ollama.chat(
    # model=""
    model="llama3.1",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant \
            that helps users craft and send their emails using the send_email tool.'},
        {'role': 'user', 'content': 'Can you send an email \
            to lucasbnsoares@hotmail.com, containing a nice motivational message?"'},
    ],
    tools=[{
        'type': 'function',
        'name': 'send_email',
        'description': 'Send an email to a user, the input params are to,\
            subject and body.',
        'parameters': {
            'type': 'object',
            'properties': {
                'to': {
                    'type': 'string',
                    'description': 'The email address of the receiver',
                },
                'subject': {
                    'type': 'string',
                    'description': 'The subject of the email',
                },
                'body': {
                    'type': 'string',
                    'description': 'The body of the email',
                }
            },
            'required': ['to', 'subject', 'body']
        }
    }]
)

print(response)

{'model': 'llama3.1', 'created_at': '2024-12-02T18:19:17.554671Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'send_email', 'arguments': {'body': 'Hello!\n\nYou got this! Keep pushing forward and never give up on your dreams.\n\nBest regards,', 'subject': '', 'to': 'lucasbnsoares@hotmail.com'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1373623292, 'load_duration': 31130709, 'prompt_eval_count': 164, 'prompt_eval_duration': 413000000, 'eval_count': 56, 'eval_duration': 927000000}


In [3]:
response

{'model': 'llama3.1',
 'created_at': '2024-12-02T18:19:17.554671Z',
 'message': {'role': 'assistant',
  'content': '',
  'tool_calls': [{'function': {'name': 'send_email',
     'arguments': {'body': 'Hello!\n\nYou got this! Keep pushing forward and never give up on your dreams.\n\nBest regards,',
      'subject': '',
      'to': 'lucasbnsoares@hotmail.com'}}}]},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 1373623292,
 'load_duration': 31130709,
 'prompt_eval_count': 164,
 'prompt_eval_duration': 413000000,
 'eval_count': 56,
 'eval_duration': 927000000}

In [4]:
response['message']['tool_calls']

[{'function': {'name': 'send_email',
   'arguments': {'body': 'Hello!\n\nYou got this! Keep pushing forward and never give up on your dreams.\n\nBest regards,',
    'subject': '',
    'to': 'lucasbnsoares@hotmail.com'}}}]

In [5]:
def run_tool_calls(response):
    tool_calls = response['message']['tool_calls']
    for tool_call in tool_calls:
        function_name = tool_call['function']['name']
        arguments = tool_call['function']['arguments']
        if function_name == 'send_email':
            to = arguments['to']
            subject = arguments['subject']
            body = arguments['body']
            send_email(to, subject, body)
            print(f"Email sent to {to} with subject: {subject} and body: {body}")
        else:
            print(f"Unknown tool call: {function_name}")

run_tool_calls(response)

Email sent successfully!
Email sent to lucasbnsoares@hotmail.com with subject:  and body: Hello!

You got this! Keep pushing forward and never give up on your dreams.

Best regards,


![](2024-12-02-18-22-41.png)

# Using OpenAI API + Ollama

In [6]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

In [7]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that helps users craft and send their emails using the send_email tool."},
    {"role": "user", "content": "Can you send an email to lucasbnsoares@hotmail.com, containing 3 things you think are essential for someone to be a life long learner?"}
]

In [8]:
tools=[{
        'type': 'function',
        'name': 'send_email',
        'description': 'Send an email to a user, the input params are to, subject and body.',
        'parameters': {
            'type': 'object',
            'properties': {
                'to': {
                    'type': 'string',
                    'description': 'The email address of the receiver',
                },
                'subject': {
                    'type': 'string',
                    'description': 'The subject of the email',
                },
                'body': {
                    'type': 'string',
                    'description': 'The body of the email',
                }
            },
            'required': ['to', 'subject', 'body']
        }
    }]



response = client.chat.completions.create(model="llama3.1", messages=messages, tools=tools)

print(response)

ChatCompletion(id='chatcmpl-993', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_sc6x4ro7', function=Function(arguments='{"body":"Curiosity, Open-Mindedness, and Self-Discipline are essential for someone to be a lifelong learner.","subject":"Essential Qualities of a Lifelong Learner","toAddress":"lucasbnsoares@hotmail.com"}', name='send_email'), type='function')]))], created=1733149581, model='llama3.1', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=67, prompt_tokens=173, total_tokens=240))


In [9]:
response

ChatCompletion(id='chatcmpl-993', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_sc6x4ro7', function=Function(arguments='{"body":"Curiosity, Open-Mindedness, and Self-Discipline are essential for someone to be a lifelong learner.","subject":"Essential Qualities of a Lifelong Learner","toAddress":"lucasbnsoares@hotmail.com"}', name='send_email'), type='function')]))], created=1733149581, model='llama3.1', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=67, prompt_tokens=173, total_tokens=240))

In [10]:
response.choices[0].message.tool_calls[0].function.arguments

'{"body":"Curiosity, Open-Mindedness, and Self-Discipline are essential for someone to be a lifelong learner.","subject":"Essential Qualities of a Lifelong Learner","toAddress":"lucasbnsoares@hotmail.com"}'